In [1]:
import json
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import datetime
import os
from pycocotools import coco
import xml.etree.ElementTree as ET
import xml

In [2]:
### CATEGORY INFORMATION
### !!! NEED HELP FOR MATCHING THE CLASSES IN VOT2016 TO THEIR CORRESPONDING ONES IN MS-COCO !!! ### 
### Legend ###
### * --> Will be defined into Mask R-CNN as a new category
### ! --> Might be some issues with the assignment, they have to be rechecked
dict_of_categories = {'face': 91}

In [3]:
json_dir = "Datasets/coco/annotations_old/instances_train2014.json"

minival2014 = json.load(open(json_dir))

minival2014.keys()

dict_keys(['info', 'images', 'licenses', 'annotations', 'categories'])

In [4]:
# list = minival2014['annotations'][604864]
# print(list['segmentation'])
# for d, i in enumerate(list):
#     if i['iscrowd']:
#         print(d)
#         print(i['segmentation'])

In [5]:
# Adds the classes, view the category list at first!!
# minival2014['categories'].append({'id': 91, 'name': 'face', 'supercategory': 'person'})
# minival2014['categories'].append({'id': 92, 'name': 'fish', 'supercategory': 'animal'})
# minival2014['categories']


In [6]:
# Shows the added class at the bottom.
# minival2014['images'].pop()
# (minival2014['images'][-20:])


In [7]:
# Some functions to be used:
def isjpg(string):
    if string[-4:]==".jpg":
        return True 
    
def ispng(string):
    if string[-4:]==".png":
        return True 
    
def DateCaptured():
    dt = str(datetime.datetime.now())
    date, time = dt.split()
    time = time.split(".")[0]
    return date+" "+time

def coco_bbox_creator(x, y):
    x = list(map(lambda x: float(x), x))
    y = list(map(lambda x: float(x), y))
    x_min = min(x)
    y_min = min(y)
    w = max(x) - x_min
    h = max(y) - y_min
    return [x_min, y_min, w, h]

def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def xml_retreiver(tree_object, key_name):
    for iterator in tree_object.iter(key_name):
        return iterator.text
    
import pycocotools._mask as _mask
encode_mask = _mask.encode

In [8]:
# LabelMe type
dataset_dir = "/home/mspr/Desktop/Mask_RCNN/Datasets/"  # Will be changed with argparse, includes directories for video names and corresponding frames inside
dataset_name = "MSPR_Dataset/"
annot_dir = "Annotations/"
mask_dir = "Masks/"
image_dir = "Images/"

target_dataset_name = os.path.join(dataset_dir, "coco/train2014/")

# Added image and annotation ids will start from 600k. 
image_id = 600000
annotation_id = 600000

for class_name in dict_of_categories.keys():
    annot_all_files = sorted(os.listdir(os.path.join(dataset_dir, dataset_name, class_name, annot_dir)))
    for xml_file in annot_all_files:
        annot_dir = os.path.join(dataset_dir, dataset_name, class_name, "Annotations/")
        mask_dir = os.path.join(dataset_dir, dataset_name, class_name, "Masks/")
        image_dir = os.path.join(dataset_dir, dataset_name, class_name, "Images/")

        tree = ET.parse(os.path.join(annot_dir, xml_file))
        root = tree.getroot()

        delete_flags = root.findall("./object/deleted")
#         bbox_x = float(xml_retreiver(root, 'xmin'))
#         bbox_y = float(xml_retreiver(root, 'ymin'))
#         bbox_w = float(xml_retreiver(root, 'xmax')) - bbox_x
#         bbox_h = float(xml_retreiver(root, 'ymax')) - bbox_y
        
        xmins = root.findall("./object/segm/box/xmin")
        ymins = root.findall("./object/segm/box/ymin")
        xmaxs = root.findall("./object/segm/box/xmax")
        ymaxs = root.findall("./object/segm/box/ymax")
        
        mask_filenames = root.findall("./object/segm/mask")

        # Polygon Handling
        
        for d, delete_flag in enumerate(delete_flags):
            delete_flag = int(delete_flag.text)
            if not delete_flag:
                # IMAGES
                # Image will be copied to MS COCO train directory and file name will be hold
                image_filename = xml_retreiver(root, 'filename')
                image = io.imread(os.path.join(image_dir, image_filename))
                if image_id<1000000:
                    image_target_filename = "COCO_train2014_000000"+str(image_id)+".jpg"
                elif (image_id>=1000000 or image_id<=10000000):
                    image_target_filename = "COCO_train2014_00000"+str(image_id)+".jpg"

                if os.path.isfile(os.path.join(target_dataset_name, image_target_filename))==False:
                    io.imsave(os.path.join(target_dataset_name, image_target_filename), image)


                # Image height and width
                height = int(xml_retreiver(root, 'nrows'))
                width = int(xml_retreiver(root, 'ncols'))

                assert height == image.shape[0]
                assert width == image.shape[1]


                # Miscellaneous metadata
                date_captured = 'n/a'
                url = 'n/a'
                license = np.random.randn(8)

                # ANNOTATIONS
                # Bbox
                bbox_x = int(xmins[d].text)
                bbox_y = int(ymins[d].text)
                bbox_w = int(xmaxs[d].text) - bbox_x
                bbox_h = int(ymaxs[d].text) - bbox_y
                
                with open("bbox_info.txt", "a+") as f:
                    f.write(image_filename[:-4]+","+str(bbox_x)+","+str(bbox_y)+","+
                            str(bbox_x+bbox_w)+","+str(bbox_y)+","+
                            str(bbox_x)+","+str(bbox_y+bbox_h)+","+
                           str(bbox_x+bbox_w)+","+str(bbox_y+bbox_h)+"\n")

                # Class
                category_id = dict_of_categories[class_name]

                # Masks
                instance_seg_info = []
                mask = io.imread(os.path.join(mask_dir, mask_filenames[d].text))[:,:,:4] 
                mask = np.asfortranarray(mask)
                instance_seg_info.append(_mask.encode(mask))   

                # Iscrowd
                if len(mask_filenames)>1:
                    iscrowd = 1
                else:
                    iscrowd = 0


#                 print(bbox_x, bbox_y, bbox_w, bbox_h)


In [17]:
dataset_dir = "/home/mspr/Desktop/Mask_RCNN/Datasets/"  # Will be changed with argparse, includes directories for video names and corresponding frames inside
train = "train/"                                                # train
dataset_seg_dir = "GT_segmentation/" # one indexed version for providing segmentation data.

source_dataset_name = os.path.join(dataset_dir, "VOT2016/" , train)
source_dataset_seg_loc = os.path.join(dataset_dir, "VOT2016/" , dataset_seg_dir)
target_dataset_name = os.path.join(dataset_dir, "coco/train2014/")

ground_truth_bbox = "groundtruth.txt"

id_counter = 600000

all_video_names = [folder for folder in os.listdir(source_dataset_name) if os.path.isdir(os.path.join(source_dataset_name, folder))] 

# Change: all_video_names --> dict_of_categories.keys()
for video_name in dict_of_categories.keys():
    all_frame_names = [frame for frame in os.listdir(os.path.join(source_dataset_name,
                                                                  video_name)) if isjpg(os.path.join(source_dataset_name, video_name, frame))]
    all_frame_names = sorted(all_frame_names)
    all_seg_frame_names = [frame for frame in os.listdir(os.path.join(source_dataset_seg_loc,
                                                                      video_name)) if ispng(os.path.join(source_dataset_seg_loc, video_name, frame))]
    all_seg_frame_names = sorted(all_seg_frame_names)
    
    # bbox
    bbox_info_file = os.path.join(source_dataset_name, video_name, ground_truth_bbox)
    with open(bbox_info_file) as f:
        all_bboxes = f.read().split()       
        
    for d, frame_name in enumerate(all_frame_names):
            
        # Date captured
        date_captured = DateCaptured()
        
        # Copying to the COCO Dataset Directory in an appropiate format.
        # Filename and ID will be collected from here.
        if id_counter<1000000:
            frame_file_name = "COCO_train2014_000000"+str(id_counter)+".jpg"
        elif (id_counter>=1000000 or id_counter<=10000000):
            frame_file_name = "COCO_train2014_00000"+str(id_counter)+".jpg"
       
        image = io.imread(os.path.join(source_dataset_name, video_name, frame_name))

        if os.path.isfile(os.path.join(target_dataset_name, frame_file_name))==False:
            io.imsave(os.path.join(target_dataset_name, frame_file_name), image)
        
        # License tag at random
        license = np.random.randint(8)
        
        # Getting image width and height
        height = image.shape[1]
        width = image.shape[0]

        # URL Information is unknown
        url = 'n/a'
        
        # Appending relevant information to images section
#         minival2014['images'].append({'date_captured': date_captured, 'file_name': frame_file_name, 
#                               'height': height, 'id': id_counter, 'license': license, 'flickr_url': url, 
#                               'width': width, 'coco_url': url})

        # bbox operations
        coords = np.array(all_bboxes[d].split(",")).reshape((-1, 2))
        x, y = coords[:,0], coords[:,1]
        xywh = coco_bbox_creator(x, y)
        
        # Category ID 
        # Create a dictionary where the keys will be video names and 
        # values will be category id's. Then calling this dictionary
        # appropiately will output the category id. 
        # E.g --> dict['fish1'] = 91
        category_id = 91
        
        # id
        annot_id = 300001

        # Is crowded?
        iscrowd = 0

        # Image ID
        image_id = id_counter

        # Segmentation
        seg_image = io.imread(os.path.join(source_dataset_seg_loc, video_name, all_seg_frame_names[d]))/255
        seg_coords_x, seg_coords_y = map(lambda x: list(x), np.where(seg_image==1))
        seg_coords = []
        for d, x in enumerate(seg_coords_x):
            seg_coords.append(x)
            seg_coords.append(seg_coords_y[d])

        # Area
        area = PolyArea(seg_coords_x, seg_coords_y)
        
        # Appending relevant info to the annotation section
#         minival2014['annotations'].append({'area': area, 'bbox': xywh, 
#                                    'category_id': category_id,
#                                    'id': annot_id, 'image_id': image_id, 'iscrowd': iscrowd, 'segmentation': [seg_coords]})
        id_counter += 1
#         print(os.path.isfile(os.path.join(dataset_dir, source_dataset_name, train, video_name+"/", frame_name)))
#         print(dataset_dir, target_dataset_name, frame_file_name)

In [24]:
# Images in segmentation directory are zero indexed and original image directory is one indexed.

# dataset_dir = "/home/mspr/Desktop/Mask_RCNN/Datasets/VOT2016/"  # Will be changed with argparse, includes directories for video names and corresponding frames inside
# train = "train/"                                                # train
# dataset_seg_dir = "GT_segmentation/" # one indexed version for providing segmentation data. 

# # Date captured
# dt = str(datetime.datetime.now())
# date, time = dt.split()
# time = time.split(".")[0]
# date_captured = date+" "+time

# # Copying to the COCO Dataset Directory in an appropiate format.
# # Filename and ID will be collected from here.
# target_dataset_dir = "Datasets/coco/train2014/"
# idx = 600001
# file_name = "COCO_train2014_000000"+str(idx)+".jpg"
# image = io.imread(image_dir)
# io.imsave(target_dataset_dir+file_name, image)

# # License tag at random
# license = np.random.randint(8)

# # Getting image width and height
# height = image.shape[1]
# width = image.shape[0]

# # URL Information is unknown
# url = 'n/a'
# # minival2014['images'].append({'date_captured': date_captured, 'file_name': file_name, 
# #                               'height': height, 'id': idx, 'license': license, 'flickr_url': url, 
# #                               'width': width, 'coco_url': url})

In [63]:
# (minival2014['images'][-3:])
# minival2014['images'].pop()

In [64]:
# Annotations

# def coco_bbox_creator(x, y):
#     x = list(map(lambda x: float(x), x))
#     y = list(map(lambda x: float(x), y))
#     x_min = min(x)
#     y_min = min(y)
#     w = max(x) - x_min
#     h = max(y) - y_min
#     return [x_min, y_min, w, h]

# # bbox
# bbox_info_file = "/home/mspr/Desktop/Mask_RCNN/Datasets/VOT2016/train/fish1/groundtruth.txt"
# with open(bbox_info_file) as f:
#     all_bboxes = f.read().split()
#     for bbox in all_bboxes[1:2]:
#         coords = np.array(bbox.split(",")).reshape((-1, 2)) 
#         x, y = coords[:,0], coords[:,1]
#         xywh = coco_bbox_creator(x, y)
#         print(xywh)

# # category id
# # Create a dictionary where the keys will be video names and 
# # values will be category id's. Then calling this dictionary
# # appropiately will output the category id. 
# # E.g --> dict['fish1'] = 91
# category_id = 91

# # id
# annot_id = 300001
    
# # Is crowded?
# iscrowd = 0

# # Image ID
# image_id = idx

# # Segmentation
# seg_image = io.imread(seg_image_dir)/255
# seg_coords_x, seg_coords_y = map(lambda x: list(x), np.where(seg_image==1))
# seg_coords = []
# for d, x in enumerate(seg_coords_x):
#     seg_coords.append(x)
#     seg_coords.append(seg_coords_y[d])

# # Area
# def PolyArea(x,y):
#     return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

# area = PolyArea(seg_coords_x, seg_coords_y)

# minival2014['annotations'].append({'area': area, 'bbox': xywh, 
#                                    'category_id': category_id,
#                                    'id': annot_id, 'image_id': idx, 'iscrowd': iscrowd, 'segmentation': [seg_coords]})

In [66]:
# minival2014['annotations'].pop()
# print (minival2014['annotations'][-1])


In [22]:
# Storing the data in memory and then writes them into a file. 

# from decimal import Decimal
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

with open("Datasets/coco/annotations/instances_minival2014.json","w") as f:
    data = json.dumps(minival2014, cls=MyEncoder)
    f.write(data)
